In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import gensim
import re, string
import math
from scipy import sparse
from scipy.sparse import csr_matrix, hstack
import sklearn
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, LinearRegression


%matplotlib inline

In [ ]:
hours = pd.read_excel('ActivitiesGangHours.xlsx', index = 'Activity ID')
#hours.drop('Activities Description', axis=1, inplace=True) #TODO: revise!!!

In [ ]:
df = pd.read_excel('ActivityConstrains.xlsx', index = 'Activity ID')
df.drop(['Name','Unnamed: 7'], axis=1,inplace=True)
df = df.dropna()###disabel if instructions how to handle provided
df["Activity ID"]=df["Activity ID"].astype(int)

In [ ]:
df.columns = ['ID','Description','Element','Extractor','Operation','Value','Unit']

mm treatment logic should be revised. Very small values are not considered in the model due to regularization.

In [ ]:
df.loc[df.Unit=='mm','Value']/=1000
df.loc[df.Unit=='mm','Unit']='m'

### Values and categories processing

Factorizing categorical features

In [ ]:
cat_df = df[df.Operation!='=']
cat_df['cat_value'] = cat_df[['Element','Extractor','Value']].groupby(by=['Element','Extractor'])['Value'].transform(lambda x: pd.factorize(x, sort=True)[0])
cat_df['cat_value']+=1 ##To leave zeros for non-interval values
cat_df.loc[cat_df['Operation'].isin(['>','>=','=>']),'cat_value']+=1

Flattening the table

In [ ]:
df['cat_value']=0
df.loc[cat_df.index,'cat_value']=cat_df['cat_value']
df.loc[cat_df.index,'Value']=0
df.Value +=df.cat_value
df.drop('cat_value',axis=1,inplace=True)

In [ ]:
### Normalize features if clustering is planned
#df['Value']/=df.Value.max()

In [ ]:
df['Op']=0
df.loc[df.Operation=='=','Op']=1
df['ElExtOp']=df.Element +  df.Extractor + df.Op.astype(str)

In [ ]:
sp = df.pivot(columns='ElExtOp', values='Value').fillna(0.0)
sp['ID']=df.ID
sp = sp.groupby(by='ID').mean().reset_index()

Excluding constant features

In [ ]:
sp = sp[sp.columns[np.nonzero(sp.replace(0.0,np.NaN).mean()!=sp.replace(0.0,np.NaN).max())]]

In [ ]:
# Add numbers extraction from description

Generating polynomial (interaction) features, but only for non-categorical data within each element

In [ ]:
elements = df.Element.unique()
cols = pd.Series(sp.columns)
pf = PolynomialFeatures(3,True,False)
c = sp['ID']
for element in elements:
    subcol_ix = np.nonzero(cols.apply(lambda x: (element in x) & ('0' in x)))[0]
    if len(subcol_ix) > 1:
        sub_cols = sp.columns[subcol_ix]
        sub_sp = sp[sub_cols]
        sub_poly = csr_matrix(pf.fit_transform(sub_sp))
        cols_poly=pf.get_feature_names(sub_cols)
        sub_df = pd.SparseDataFrame(sub_poly, columns=cols_poly)
        c= pd.concat([c,sub_df],axis=1)
sp=c.fillna(0.0)

### Description processing

In [ ]:
### extract numbers here

In [ ]:
regex = re.compile('[' +re.escape(string.punctuation) + '\d\.\\r\\t\\n]')
descr = df['Description'].apply(lambda x: regex.sub(" ",x))

In [ ]:
stopwords = df.Extractor.unique().tolist()
stopwords += df.Element.unique().tolist()
stopwords += df.Unit.unique().tolist()
stopwords += stop_words.ENGLISH_STOP_WORDS
stopwords += ['exceeding','over','thickness', 'size','like','mm']
len(stopwords)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word',stop_words=frozenset(stopwords))
vectorizer.fit(descr)

In [ ]:
descr_df = pd.DataFrame(vectorizer.transform(descr).todense(),columns=list(vectorizer.vocabulary_.keys())).fillna(0.0)

In [ ]:
descr_df.index = df.index
descr_df['ID']=df.ID.astype(int)
descr_df = descr_df.groupby(by='ID').mean().reset_index()

In [ ]:
descr_df.shape

### Merging and getting polynomial features

In [ ]:
ds = descr_df.merge(sp,on='ID',how='left')

In [ ]:
ds.to_csv('table.csv', index=False) ###To start from the preprocessed data

In [ ]:
ds = pd.read_csv('table.csv')

In [ ]:
y = np.log(ds.merge(hours, on='ID', how='left')['Gang Hours'])

On a histogram of working hours we can see that there are a lot of values with the same value, which is a minimum value of the variable and it has no continuous link to the rest of the distribution

In [ ]:
pd.Series(y).hist(bins=300)
(y==y.min()).sum()

Let's check if there's any features which are present only for such records

In [ ]:
strange_cols  = ds.columns[np.nonzero(ds[y!=y.min].sum()==0.0)[0]]
strange_cols

Unfortunately there're no unique words in description, so we could identify such records even before priliminary parsing. Domain knowledge could help to explain are these values valid or not. For valid case we could simply create a rule without any learning. Anyway let's take them away from the train set with features.

In [ ]:
y[y==y.min()]=None
ds.drop(strange_cols,axis=1, inplace=True)

For a linear model we need to create features multiplying the word vectors by the feature values. Standard sklearn PolynomialFeatures will create a lot of excessive ones, so we need to write the method for this.

In [ ]:
def words_values_interactions(df, word_cols, values_cols):
    d = df.copy()
    for word_col in word_cols:
        for value_col in values_cols:
            col = ' '.join([word_col,value_col])
            d[col]= df[word_col] * df[value_col]
    return d

In [ ]:
value_cols = ds.columns[np.nonzero(pd.Series(ds.columns).apply(lambda x: ('1' in x) | ('0' in x)))[0]]
word_cols = list((set(ds.columns)-set(value_cols))-set('ID'))
word_cols.remove('ID')
d = words_values_interactions(ds,word_cols,value_cols)

In [ ]:
r = Ridge(alpha=0.01)
r.fit(d[~y.isnull()].drop('ID',axis=1).fillna(0.0),y.dropna())

In [ ]:
y_pred = r.predict(d[~y.isnull()].drop('ID',axis=1).fillna(0.0))
metrics.mean_squared_error(y_pred, y.dropna())

Influential features

In [ ]:
d.columns[np.argsort(r.coef_)[-20:]]

In [ ]:
worst = np.argsort(np.abs((y_pred - y.dropna())/y.dropna()))[-20:]
hours.copy().reindex(worst)[['ID','Activities Description']]
